In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# 1. Combine frauds data with NonFraud data

In [2]:
frauds = pd.read_csv('csv files/all_frauds_by_filing_and_tags.csv')
qtrs=sorted(frauds.related_qtr.unique().tolist())
len(qtrs) #missing 2q16, will import that df

35

In [3]:
frauds.head()

,adsh,comp_name,related_qtr,filing_type,fraud,tag,version,ddate,qtrs,uom,value
0,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccountsPayableCurrent,us-gaap/2009,20090331.0,0.0,USD,6.360000e+08
1,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccountsPayableCurrent,us-gaap/2009,20100331.0,0.0,USD,4.090000e+08
2,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccruedLiabilitiesCurrent,us-gaap/2009,20090331.0,0.0,USD,1.264000e+09
3,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccruedLiabilitiesCurrent,us-gaap/2009,20100331.0,0.0,USD,1.344000e+09
4,0000023082-10-000036,COMPUTER SCIENCES CORP,2q10,10k,1,AccumulatedDepreciationDepletionAndAmortizatio...,us-gaap/2009,20090331.0,0.0,USD,-3.417000e+09


In [4]:
frauds.isnull().sum()

adsh            0
comp_name       0
related_qtr     0
filing_type     0
fraud           0
tag            62
version        62
ddate          62
qtrs           62
uom            62
value          62
dtype: int64

In [5]:
frauds = frauds.dropna()

In [7]:
df = pd.read_csv('csv files/2018q1/num.txt', sep='\t',encoding ='latin1')
df.head()

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001683168-18-000002,EntityPublicFloat,dei/2014,NaN,20160630,0,USD,3200000.0,NaN
1,0001562762-17-000231,EntityPublicFloat,dei/2014,NaN,20170331,0,USD,28500000.0,NaN
2,0001599916-18-000002,EntityPublicFloat,dei/2014,NaN,20171231,0,USD,253660.0,NaN
3,0001614106-18-000003,EntityPublicFloat,dei/2014,NaN,20161231,0,USD,18247.0,NaN
4,0001144204-18-000327,EntityPublicFloat,dei/2014,NaN,20170331,0,USD,23466799.0,NaN


In [8]:
df.shape

(3030131, 9)

In [9]:
df.isnull().sum()

adsh              0
tag               0
version           0
coreg       2865204
ddate             0
qtrs              0
uom               0
value         50173
footnote    3023631
dtype: int64

In [10]:
df = df.drop(columns=['footnote','coreg'])
df = df.dropna()
df.shape

(2979958, 7)

In [11]:
testfrauds = frauds['adsh']
dftest = df['adsh']

In [12]:
# testfrauds = testfrauds.set_index(['adsh'])
# dftest = dftest.set_index(['adsh'])

In [13]:
dftest.head()

0    0001683168-18-000002
1    0001562762-17-000231
2    0001599916-18-000002
3    0001614106-18-000003
4    0001144204-18-000327
Name: adsh, dtype: object

In [14]:
testfrauds.head()

0    0000023082-10-000036
1    0000023082-10-000036
2    0000023082-10-000036
3    0000023082-10-000036
4    0000023082-10-000036
Name: adsh, dtype: object

In [15]:
result = dftest[dftest.isin(testfrauds)]
result

Series([], Name: adsh, dtype: object)

In [16]:
combined = frauds.append(df)

In [17]:
combined.shape

(3040863, 11)

In [19]:
combined = combined.drop(columns=['comp_name','filing_type','related_qtr'])

In [20]:
combined.isnull().sum()

adsh             0
ddate            0
fraud      2979958
qtrs             0
tag              0
uom              0
value            0
version          0
dtype: int64

In [21]:
combined = combined.fillna(0)

In [22]:
combined.isnull().sum()

adsh       0
ddate      0
fraud      0
qtrs       0
tag        0
uom        0
value      0
version    0
dtype: int64

In [23]:
combined.to_csv('06.a.combined_fraud_nonfraud_data_with_tags.csv')

# 2. Create Tag buckets

In [132]:
combined = pd.read_csv('06.a.combined_fraud_nonfraud_data_with_tags.csv')

In [125]:
combined.head()

,Unnamed: 0,adsh,ddate,fraud,qtrs,tag,uom,value,version
0,0,0000023082-10-000036,20090331.0,1.0,0.0,AccountsPayableCurrent,USD,6.360000e+08,us-gaap/2009
1,1,0000023082-10-000036,20100331.0,1.0,0.0,AccountsPayableCurrent,USD,4.090000e+08,us-gaap/2009
2,2,0000023082-10-000036,20090331.0,1.0,0.0,AccruedLiabilitiesCurrent,USD,1.264000e+09,us-gaap/2009
3,3,0000023082-10-000036,20100331.0,1.0,0.0,AccruedLiabilitiesCurrent,USD,1.344000e+09,us-gaap/2009
4,4,0000023082-10-000036,20090331.0,1.0,0.0,AccumulatedDepreciationDepletionAndAmortizatio...,USD,-3.417000e+09,us-gaap/2009


In [133]:
combined['tag'] = combined['tag'].str.lower()
combined['2ndindex'] = combined['tag']
combined = combined.set_index(['adsh', '2ndindex'])
combined = combined.loc[combined['uom'] == 'USD']
alltags = combined.tag.unique().tolist()
print(len(alltags))
# for i in allfraudtags:
#     print(i, len(frauds.loc[frauds['tag'] == i ]))

132350


# Final Bucketing Order


In [134]:
# CashFlowfromOps
combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains("financingactivit|financeact|noncash")].
                                            index, 'tag', 'NotCash_Catch')

op_ativities_cash = "operatingactivi|operationactivit|operatingcash"

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(op_ativities_cash)].
                                            index, 'tag', 'CashFlowfromOps')

#Depreciation
combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains('depreciation')].
                                            index, 'tag', 'Depreciation')
# intangibles
intangibles = 'goodwill|intangible|trademark|royalt|software|patent|copyright|license|customerlist'
combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(intangibles)].
                                            index, 'tag', 'IntangibleAssets')
#AccountsReceivable, OtherCurrentAssets
accounts_rec = "accountsrec|accountrec|receivable|uncollectible|allowancefor"
other_current_assets = "inventor|asset|taxben|prepaid|fairval|taxexpensebenefit"

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(accounts_rec)].
                                            index, 'tag', 'AccountsReceivable')

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(other_current_assets)].
                                            index, 'tag', 'OtherCurrentAssets')

#PPE,
PPE = "land|properties|property|building|leaseh|lease-h|rental|equipment|machinery|longtermasset"

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(PPE)].
                                            index, 'tag', 'PPE')


#currentliabilities,other liabilities
current_liabilities = "payable|short-term|currentliab|currentmatur|currentport|accrued|deferredinc"



combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(current_liabilities)].
                                            index, 'tag', 'CurrentLiabilities')

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains("salesecurities")].
                                            index, 'tag', 'NotSales_Catch')


#sales, expenses#otherincome
sales = "revenue|sales"

expenses = "costof|taxespaid|services|costsof|research|expense|admin|cost|compensat|charge|settlement|deplet|accretion|amortiz"


other_income = "proceeds"

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(sales)].
                                            index, 'tag', 'Sales')

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(expenses)].
                                            index, 'tag', 'Expenses')

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(other_income)].
                                            index, 'tag', 'OtherIncome')

#Cash
cash = "cash|cashequiv|commercialpaper|cashand"
combined = combined.set_value(combined.loc[combined['tag'].str.contains(cash)].
                                            index, 'tag', 'Cash')

#shareholdersequity
shareholders = "othercompre|shareholder|paid-in|capital|controlling|equity|retained|orityinterest|stock|accumulateddeficit"

combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(shareholders)].
                                            index, 'tag', 'ShareholdersEquity')
other_liabilities = 'long-term|debt|liabilit|lineofcredit|provision|contingen'
combined = combined.set_value(combined.loc[combined['tag'].
                                                                str.contains(other_liabilities)].
                                            index, 'tag', 'OtherLiabilities')


In [135]:
alltags = combined.tag.unique().tolist()
print(len(alltags))

34378


In [82]:
# alltags

#Depreciation -some incorrect
#CurrentLiabilities' #deferredincometaxexpensebenefit	
# 'PPE', - put depreciation before PPE, got rid of investment, changed lease to lease- h    
#  'ShareholdersEquity', -generally speaking looks good
#  'AccountsReceivable', - no issues, best predicter
#  'OtherCurrentAssets' -was capturing intangible assets so put intangible tag first,capturing otherassetsnoncurrent	
#  'IntangibleAssets' - looks good
#  'Cash', - added a not cash catch before cash set_value
#  'Expenses', - incometaxexpensebenefit: added taxexpensebenefit to othercurrent assets
#  'OtherLiabilities',liabilitiesandstockholdersequity,proceedsfromissuanceoflongtermdebt
#  'Sales'#  - looks good
#'CashFlowfromOps- looks just okay
#OtherIncome - got rid of otherincome and other incomeloss, was too specific (70047)


In [131]:
# combined.loc[lambda combined: combined['tag'] == 'CashFlowfromOps']

Unnamed: 0  \
adsh                 2ndindex                                                         
0000023082-10-000036 excesstaxbenefitfromsharebasedcompensationopera...          64   
                     excesstaxbenefitfromsharebasedcompensationopera...          65   
                     excesstaxbenefitfromsharebasedcompensationopera...          66   
                     netcashprovidedbyusedinoperatingactivities                 129   
                     netcashprovidedbyusedinoperatingactivities                 130   
                     netcashprovidedbyusedinoperatingactivities                 131   
0000023082-10-000054 netcashprovidedbyusedinoperatingactivities                 407   
                     netcashprovidedbyusedinoperatingactivities                 408   
0000023082-10-000072 netcashprovidedbyusedinoperatingactivities                 760   
                     netcashprovidedbyusedinoperatingactivities                 761   
0000023082-11-000014 netcashprovidedbyusedinoperatingactivities                1183   
                     netcashprovidedbyusedinoperatingactivities                1184   
0000023082-11-000059 excesstaxbenefitfromsharebasedcompensationopera...        1653   
                     excesstaxbenefitfromsharebasedcompensationopera...        1654   
                     excesstaxbenefitfromsharebasedcompensationopera...        1655   
                     netcashprovidedbyusedinoperatingactivities                1805   
                     netcashprovidedbyusedinoperatingactivities                1806   
                     netcashprovidedbyusedinoperatingactivities                1807   
0000023082-12-000039 excesstaxbenefitfromsharebasedcompensationopera...        2386   
                     excesstaxbenefitfromsharebasedcompensationopera...        2387   
                     excesstaxbenefitfromsharebasedcompensationopera...        2388   
                     netcashprovidedbyusedinoperatingactivities                2547   
                     netcashprovidedbyusedinoperatingactivities                2548   
                     netcashprovidedbyusedinoperatingactivities                2549   
0000023082-12-000050 netcashprovidedbyusedinoperatingactivities                3088   
                     netcashprovidedbyusedinoperatingactivities                3089   
0000099771-12-000004 excesstaxbenefitfromsharebasedcompensationopera...        3322   
                     excesstaxbenefitfromsharebasedcompensationopera...        3323   
                     excesstaxbenefitfromsharebasedcompensationopera...        3324   
                     netcashprovidedbyusedinoperatingactivities                3470   
...                                                                             ...   
0000880224-18-000016 contributionstodefinedbenefitsplanspaidclassifi...     3003249   
                     contributionstodefinedbenefitsplanspaidclassifi...     3003250   
                     profitlossfromoperatingactivitiesbeforeexpenseo...     3003326   
                     profitlossfromoperatingactivitiesbeforeexpenseo...     3003327   
                     profitlossfromoperatingactivitiesbeforeexpenseo...     3003328   
0001437749-18-005798 paymentsforpurchaseoftradingsecuritiesoperating...     3006094   
                     paymentsforpurchaseoftradingsecuritiesoperating...     3006095   
                     proceedsfromsaleoftradingsecuritiesoperatingact...     3006100   
                     proceedsfromsaleoftradingsecuritiesoperatingact...     3006101   
0001615774-18-002185 operatingcashaccountsatcarryingvalue                   3006739   
                     operatingcashaccountsatcarryingvalue                   3006740   
0001144204-18-017931 adjustmentsnoncashitemstoreconcilenetincomeloss...     3007692   
                     adjustmentsnoncashitemstoreconcilenetincomeloss...     3007693   
                     adjustmentsnoncashitemstoreconcilenetincomeloss...

In [136]:
combined.to_csv('06.combined_df_bucketed_specific_13_tags.csv')

In [143]:
bucket_tags = ['Depreciation', 'CurrentLiabilities' , 'PPE', 'ShareholdersEquity','AccountsReceivable', 
               'OtherCurrentAssets','IntangibleAssets','Cash','Expenses','OtherLiabilities','Sales',
               'CashFlowfromOps','OtherIncome']

In [140]:
bucket_df = combined[combined['tag'].isin(['Depreciation', 'CurrentLiabilities' , 'PPE', 'ShareholdersEquity','AccountsReceivable', 
               'OtherCurrentAssets','IntangibleAssets','Cash','Expenses','OtherLiabilities','Sales',
               'CashFlowfromOps','OtherIncome'])]
                                            

In [145]:
for i in bucket_tags:
    print(i, len(combined.loc[combined['tag'] == i ]))

Depreciation 44936
CurrentLiabilities 105753
PPE 101515
ShareholdersEquity 367695
AccountsReceivable 103530
OtherCurrentAssets 431284
IntangibleAssets 87554
Cash 103168
Expenses 350768
OtherLiabilities 156649
Sales 81484
CashFlowfromOps 24247
OtherIncome 62724


In [146]:
len(bucket_df.index.unique())


835467

In [147]:
len(df.adsh.unique())

5992

In [148]:
bucket_df.to_csv('combined_df_filtered_for_13_tags.csv')

In [159]:
bucket_df= pd.read_csv('combined_df_filtered_for_13_tags.csv')
bucket_df.head()

,adsh,2ndindex,Unnamed: 0,ddate,fraud,qtrs,tag,uom,value,version
0,0000023082-10-000036,accountspayablecurrent,0,20090331.0,1.0,0.0,CurrentLiabilities,USD,6.360000e+08,us-gaap/2009
1,0000023082-10-000036,accountspayablecurrent,1,20100331.0,1.0,0.0,CurrentLiabilities,USD,4.090000e+08,us-gaap/2009
2,0000023082-10-000036,accruedliabilitiescurrent,2,20090331.0,1.0,0.0,CurrentLiabilities,USD,1.264000e+09,us-gaap/2009
3,0000023082-10-000036,accruedliabilitiescurrent,3,20100331.0,1.0,0.0,CurrentLiabilities,USD,1.344000e+09,us-gaap/2009
4,0000023082-10-000036,accumulateddepreciationdepletionandamortizatio...,4,20090331.0,1.0,0.0,Depreciation,USD,-3.417000e+09,us-gaap/2009


In [160]:
fraudsters = bucket_df.loc[bucket_df.fraud == 1]
len(fraudsters.adsh.unique())


121

In [161]:
nonfrauds = bucket_df.loc[bucket_df.fraud == 0]
len(nonfrauds.adsh.unique())

5911

In [163]:
df_for_pivot = bucket_df.drop(columns= ['2ndindex', 'qtrs', 'uom','version'])
df_for_pivot.head()

,adsh,Unnamed: 0,ddate,fraud,tag,value
0,0000023082-10-000036,0,20090331.0,1.0,CurrentLiabilities,6.360000e+08
1,0000023082-10-000036,1,20100331.0,1.0,CurrentLiabilities,4.090000e+08
2,0000023082-10-000036,2,20090331.0,1.0,CurrentLiabilities,1.264000e+09
3,0000023082-10-000036,3,20100331.0,1.0,CurrentLiabilities,1.344000e+09
4,0000023082-10-000036,4,20090331.0,1.0,Depreciation,-3.417000e+09


In [165]:
# bucket_tag_sum = 0
# for i in bucket_tags:
#     combined_pivot[i].sum() += bucket_tag_sum
# bucket_tag_sum


In [166]:
combined_pivot =pd.pivot_table(df_for_pivot,index=['fraud',"adsh","ddate",], columns= ['tag'], values= ['value'], aggfunc='sum')

In [169]:
combined_pivot = combined_pivot.fillna(0)
combined_pivot

value                \
tag                                   AccountsReceivable          Cash   
fraud adsh                 ddate                                         
0.0   0000002178-18-000009 20141231.0       1.790000e+05  8.018400e+07   
                           20151231.0      -7.068200e+07  9.721000e+07   
                           20160331.0       0.000000e+00  0.000000e+00   
                           20160630.0       0.000000e+00  0.000000e+00   
                           20160930.0       0.000000e+00  0.000000e+00   
                           20161231.0       1.067560e+08  7.875000e+07   
                           20170331.0       0.000000e+00  0.000000e+00   
                           20170430.0       0.000000e+00  0.000000e+00   
                           20170531.0       0.000000e+00  0.000000e+00   
                           20170630.0       0.000000e+00  0.000000e+00   
                           20170831.0       0.000000e+00  0.000000e+00   
                           20170930.0       0.000000e+00  0.000000e+00   
                           20171231.0       1.576660e+08  1.349390e+08   
      0000002488-18-000042 20141231.0       0.000000e+00  8.050000e+08   
                           20151231.0      -2.800000e+08  9.120000e+08   
                           20160229.0       0.000000e+00  0.000000e+00   
                           20160331.0       0.000000e+00  0.000000e+00   
                           20160630.0       0.000000e+00  0.000000e+00   
                           20160831.0       0.000000e+00  0.000000e+00   
                           20160930.0       0.000000e+00  0.000000e+00   
                           20161231.0       1.260000e+08  2.010000e+09   
                           20170331.0       0.000000e+00  0.000000e+00   
                           20170630.0       0.000000e+00  0.000000e+00   
                           20170930.0       0.000000e+00  0.000000e+00   
                           20171231.0       5.210000e+08  9.920000e+08   
      0000002969-18-000014 20160930.0       0.000000e+00  1.330800e+09   
                           20161231.0      -5.740000e+07  5.735000e+08   
                           20170930.0       1.174000e+09  6.547200e+09   
                           20171231.0       1.220500e+09  4.879300e+09   
      0000003499-18-000006 20141231.0       0.000000e+00  6.248340e+08   
...                                                  ...           ...   
1.0   0001193125-16-701058 20160630.0       4.861950e+09  1.589071e+09   
      0001193125-16-759676 20140930.0       0.000000e+00  0.000000e+00   
                           20141231.0       0.000000e+00  8.035220e+08   
                           20150930.0       4.298000e+06  2.717552e+09   
                           20151231.0       5.673643e+09  1.102244e+09   
                           20160430.0       0.000000e+00  0.000000e+00   
                           20160531.0       0.000000e+00  1.500000e+01   
                           20160731.0       0.000000e+00  1.500000e+01   
                           20160831.0       0.000000e+00  0.000000e+00   
                           20160930.0       5.560737e+09  1.976719e+09   
      0001354488-11-003677 20100430.0       0.000000e+00  2.994634e+06   
                           20100731.0       5.328840e+05 -5.601649e+06   
                           20110430.0       5.267878e+06  1.334736e+07   
                           20110731.0       5.267878e+06  2.170679e+06   
      0001354488-11-004960 20100430.0       0.000000e+00  2.994634e+06   
                           20101031.0      -2.339250e+05 -6.175557e+06   
                           20110430.0       5.267878e+06  1.334736e+07   
                           20111031.0       9.105358e+06 -1.743344e+07   
      0001354488-12-001090 20100430.0       0.000000e+00  2.994634e+06   
                           20110131.0      -4.621000e+03 -6.511487e+06   
                           20110430.0       5.267878e+0

In [170]:
combined_pivot.to_csv('combined_pivot_table_filtered_for_13_tags.csv')
# first_model= pd.read_csv('first_model_data.csv')

In [54]:
# op_ativities_cash = "operatingact|netcash"
# combined = combined.set_value(combined.loc[combined['tag'].
# #                                                                 str.contains(op_ativities_cash)].
#                                             index, 'tag', 'CashFlowfromOps')


# Bucketing the CurrentAsset Tags

In [46]:
# cash = "cash|cashequiv|commercialpaper|cashand"
# accounts_rec = "accountsrec|accountrec|receivable|uncollectible|allowancefor"
# other_current_assets = "inventor|currentasset|taxben|prepaid|fairval"


# combined = combined.set_value(combined.loc[combined['tag'].str.contains(cash)].
#                                             index, 'tag', 'Cash')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(accounts_rec)].
#                                             index, 'tag', 'AccountsReceivable')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(other_current_assets)].
#                                             index, 'tag', 'OtherCurrentAssets')


# alltags = combined.tag.unique().tolist()
# print(len(alltags))

109829


# Bucketing Other Assets Tags

In [47]:
# PPE = "land|properties|property|building|lease|rental|equipment|machinery|investment|longtermasset"

# intangibles = 'goodwill|intangible|trademark|royalt|software|patent|copyright|license|customerlist'
# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(PPE)].
#                                             index, 'tag', 'PPE')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(intangibles)].
#                                             index, 'tag', 'IntangibleAssets')

# alltags = combined.tag.unique().tolist()
# print(len(alltags))


91020


# Bucketing Liabilities

In [48]:
# current_liabilities = "payable|short-term|currentliab|currentmatur|currentport|accrued|deferredinc"

# other_liabilities = 'long-term|debt|liabilit|lineofcredit|provision|contingen'

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(current_liabilities)].
#                                             index, 'tag', 'CurrentLiabilities')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(other_liabilities)].
#                                             index, 'tag', 'OtherLiabilities')

# alltags = combined.tag.unique().tolist()
# print(len(alltags))

70950


# Bucketing Revenue and Operating Expenses

In [49]:
# sales = "revenue|sales"

# expenses = "costof|services|costsof|research|expense|services|admin|cost|paid|compensat|charge|settlement|deplet|accretion|amortiz"

# other_income = "otherincome|otherincomeloss|proceeds"

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(sales)].
#                                             index, 'tag', 'Sales')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(expenses)].
#                                             index, 'tag', 'Expenses')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(other_income)].
#                                             index, 'tag', 'OtherIncome')

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains('depreciation')].
#                                             index, 'tag', 'Depreciation')

# alltags = combined.tag.unique().tolist()
# print(len(alltags))

45355


# Bucketing Shareholders Equity

In [50]:
# shareholders = "othercompre|shareholder|paid-in|capital|controlling|equity|retained|stock|accumulateddeficit"

# combined = combined.set_value(combined.loc[combined['tag'].
#                                                                 str.contains(shareholders)].
#                                             index, 'tag', 'ShareholdersEquity')

